# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot

map = city_data_df.hvplot.points("Lng","Lat", geo = True,tiles="OSM", color="City", size="Humidity",scale=0.8)

# Display the map

map

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

vacation_df = city_data_df.loc[(city_data_df["Max Temp"] < 26) & (city_data_df["Wind Speed"] < 7) & (city_data_df["Cloudiness"] <20) ,:]

# Drop any rows with null values

ideal_vacation_df = vacation_df.dropna(how="any")

# Display sample data

ideal_vacation_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
42,42,margate,51.3813,1.3862,16.84,76,3,1.34,GB,1666108256
43,43,ancud,-41.8697,-73.8203,14.75,70,11,2.68,CL,1666108257
44,44,collierville,35.0420,-89.6645,7.11,47,0,5.66,US,1666108151


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = pd.DataFrame().assign(City= ideal_vacation_df["City"], Country= ideal_vacation_df["Country"], Lat= ideal_vacation_df["Lat"],Lng= ideal_vacation_df["Lng"],Humidity= ideal_vacation_df["Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"]= ""

# Display sample data

hotel_df.sample(17)


,City,Country,Lat,Lng,Humidity,Hotel Name
303,alice springs,AU,-23.7000,133.8833,63,
153,lavrentiya,RU,65.5833,-171.0000,75,
240,tikaitnagar,IN,26.9500,81.5833,59,
381,roebourne,AU,-20.7833,117.1333,65,
45,kapaa,US,22.0752,-159.3190,84,
318,harwich,GB,51.9419,1.2844,69,
516,guerrero negro,MX,27.9769,-114.0611,68,
96,zhangye,CN,38.9342,100.4517,16,
397,carlyle,US,38.6103,-89.3726,62,
174,avera,US,33.1940,-82.5271,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel

radius = 10000
geoapify_key = "082579dd54374c5f9abb56810c75dc93"
params = {"apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame

for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = params = {
    'param1': 'value1',
    'param2': 'value2',
    }
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params = params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
new norfolk - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
margate - nearest hotel: No hotel found
ancud - nearest hotel: No hotel found
collierville - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
mogadishu - nearest hotel: No hotel found
hobart - nearest hotel: No hotel found
lasa - nearest hotel: No hotel found
busselton - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
qostanay - nearest hotel: No hotel found
esperance - nearest hotel: No hotel found
solnechnyy - nearest hotel: No hotel found
saint george - nearest hotel: No hotel found
zhangye - nearest hotel: No hotel found
heihe - nearest hotel: No hotel found
cedar city - nearest hotel: No hotel found
lichuan - nearest hotel: No hotel found
jashpurnagar - nearest hotel: 

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot

map_final = hotel_df.hvplot.points("Lng","Lat", geo = True,tiles="ESRI",ylim=(-70, 60), color="City", hover_cols=["City","Country","Hotel Name","Humidity"])


# Display the map

map_final

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.